# Basic usage Array1DScanAnalyzer

This notebook demonstrates the basic intended usage of Array1DScanAnalyzer. This object is meant as a compliment to ImageAnalysis to manage and perform ImageAnalysis functionality across a scan. It includes some basic functionality to perofrm the analysis in multi processing or multi threading.

The result of the using Array1DScanAnalyzer is that the analysis results from ImageAnalysis are added to the sFile and it produces some rendered results. For a 1D scan, images are averaged and binned while for 'noscan' a gif is created. These rendered results are added to the 'analysis' directory along with the averaged images by Bin stored in .hdf5 format. 

Array2DScanAnalyzer also exposes the rendered results such that they can be automatically added to the elogs generated by Logmaker4GoogleDocs. This does require that the machine running the code has full credentialed access to the shared drives. 

In [1]:
import logging

from geecs_data_utils import ScanPaths, ScanTag
from scan_analysis.base import ScanAnalyzerInfo as Info
from scan_analysis.execute_scan_analysis import instantiate_scan_analyzer
from scan_analysis.analyzers.common.array1d_scan_analysis import Array1DScanAnalyzer

# import the desired ImageAnalyzer
from image_analysis.offline_analyzers.standard_1d_analyzer import Standard1DAnalyzer

# BeamAnalyzer uses a configuration file for instantiation and use. So,
# we need to set the config path. But, any type of ImageAnalzyer can be used.
from image_analysis.config_loader import set_config_base_dir

# Set the logging level. Can set to INFO or DEBUG to get more detail about the
# actual execution
logging.getLogger("image_analysis").setLevel(logging.WARNING)
logging.getLogger("scan_analysis").setLevel(logging.WARNING)
logging.getLogger("geecs_data_utils").setLevel(logging.WARNING)
logging.getLogger("logmaker_4_googledocs").setLevel(logging.ERROR)


set_config_base_dir(ScanPaths.paths_config.image_analysis_configs_path)

2025-10-09 18:58:17,269 - geecs_data_utils.geecs_paths_config - WARNING - Z:/data path was not found
2025-10-09 18:58:18,090 - scan_analysis.execute_scan_analysis - WARNING - Docgen import failed ('script'); disabling.


Failed to load config file from: /Users/samuelbarber/Desktop/Github_repos/GEECS-Plugins/LogMaker4GoogleDocs/logmaker_4_googledocs/config.ini


Instantiate the ImageAnalyzer. Note, here the device name is also used to point to a configuration yaml file found in teh 'config_base_dir' set above

In [2]:
import time

t0 = time.monotonic()

# ScanAnalyzers use ScanTag as an arg, which is a unique identifier, to
# determine which scan to process
year = 2025
month = 9
day = 24
number = 8
experiment = "Undulator"

test_tag = ScanTag(
    year=year, month=month, day=day, number=number, experiment=experiment
)

dev_name = "U_BCaveICT"
image_analyzer = Standard1DAnalyzer(line_config_name=dev_name)


# Create an scan_analyzer_info which is used to directly instantiate the
# ScanAnalyzer (which is an Array2DScanAnalyzer in this case)
scan_analyzer_info = Info(
    scan_analyzer_class=Array1DScanAnalyzer,
    requirements={dev_name},
    device_name=dev_name,
    scan_analyzer_kwargs={
        "image_analyzer": image_analyzer,
        "file_tail": ".tdms",
        "renderer_kwargs": {
            "colormap_mode": "diverging",  # For scope traces with +/- signals
            "cmap": "RdBu_r",
        },
    },
)

scan_analyzer = instantiate_scan_analyzer(scan_analyzer_info=scan_analyzer_info)
scan_analyzer.flag_save_data = False

# run the analysis
scan_analyzer.run_analysis(scan_tag=test_tag)

t1 = time.monotonic()
print(f"execution time: {t1 - t0}")

2025-10-09 18:58:18,899 - root - INFO - analysis path is : /Volumes/hdna2/data/Undulator/Y2025/09-Sep/25_0924/analysis/Scan008
2025-10-09 18:58:18,981 - root - INFO - Scan parameter is: U_ESP_JetXYZ Position.Axis 1.
2025-10-09 18:59:03,113 - scan_analysis.analyzers.common.single_device_scan_analyzer - WARNING - Lineouts do not have expected shape for overlays
2025-10-09 18:59:03,114 - scan_analysis.analyzers.common.single_device_scan_analyzer - WARNING - Lineouts do not have expected shape for overlays
2025-10-09 18:59:03,115 - scan_analysis.analyzers.common.single_device_scan_analyzer - WARNING - Lineouts do not have expected shape for overlays
2025-10-09 18:59:03,116 - scan_analysis.analyzers.common.single_device_scan_analyzer - WARNING - Lineouts do not have expected shape for overlays
2025-10-09 18:59:03,117 - scan_analysis.analyzers.common.single_device_scan_analyzer - WARNING - Lineouts do not have expected shape for overlays
2025-10-09 18:59:03,118 - scan_analysis.analyzers.comm

execution time: 44.62996529089287


When the execution is complete, we can inspect indivdiual images

In [ ]:
dev_name = "U_BCaveMagSpec-interpSpec"
image_analyzer_mag = Standard1DAnalyzer(line_config_name=dev_name)

scan_analyzer_info_mag = Info(
    scan_analyzer_class=Array1DScanAnalyzer,
    requirements={dev_name},
    device_name=dev_name,
    scan_analyzer_kwargs={
        "image_analyzer": image_analyzer_mag,
        "file_tail": ".txt",
    },
)

scan_analyzer_mag = instantiate_scan_analyzer(scan_analyzer_info=scan_analyzer_info_mag)

# run the analysis
results = scan_analyzer_mag.run_analysis(scan_tag=test_tag)

In [3]:
from image_analysis.offline_analyzers.Undulator.hi_res_mag_cam_analyzer import (
    HiResMagCamAnalyzer,
)
from scan_analysis.analyzers.common.array2D_scan_analysis import Array2DScanAnalyzer


analyzer_info = Info(
    scan_analyzer_class=Array2DScanAnalyzer,
    requirements={"UC_HiResMagCam"},
    device_name="UC_HiResMagCam",
    scan_analyzer_kwargs={
        "image_analyzer": HiResMagCamAnalyzer(camera_config_name="UC_HiResMagCam")
    },
)

test_tag = ScanTag(year=2025, month=8, day=19, number=2, experiment="Undulator")

test_analyzer = instantiate_scan_analyzer(analyzer_info)

test_analyzer.run_analysis(scan_tag=test_tag)

In [ ]:
from image_analysis.offline_analyzers.density_from_phase_analysis import (
    PhaseAnalysisConfig,
    PhaseDownrampProcessor,
)
from scan_analysis.analyzers.common.array2D_scan_analysis import Array2DScanAnalyzer

from geecs_data_utils import ScanData, ScanTag
from dataclasses import asdict


def get_path_to_bkg_file():
    """Get path to bkd."""
    st = ScanTag(year=2025, month=3, day=6, number=15, experiment="Undulator")
    sp = ScanPaths(tag=st)
    s_data = ScanData(paths=sp)
    path_to_file = s_data.paths.get_folder() / "U_HasoLift" / "average_phase.tsv"

    return path_to_file


bkg_file_path = get_path_to_bkg_file()
config: PhaseAnalysisConfig = PhaseAnalysisConfig(
    pixel_scale=10.1,  # um per pixel (vertical)
    wavelength_nm=800,  # Probe laser wavelength in nm
    threshold_fraction=0.05,  # Threshold fraction for pre-processing
    roi=(10, -10, 75, -250),  # Example ROI: (x_min, x_max, y_min, y_max)
    background_path=bkg_file_path,  # Background is now a Path
)
config_dict = asdict(config)
analyzer_info = Info(
    scan_analyzer_class=Array2DScanAnalyzer,
    requirements={"U_HasoLift"},
    device_name="U_HasoLift",
    scan_analyzer_kwargs={
        "image_analyzer": PhaseDownrampProcessor(**config_dict),
        "file_tail": "_postprocessed.tsv",
    },
)

test_analyzer = instantiate_scan_analyzer(analyzer_info)

test_tag = ScanTag(year=2025, month=3, day=6, number=16, experiment="Undulator")
test_analyzer.run_analysis(scan_tag=test_tag)